In [3]:
"""
Loaded data from extended ion tests
"""

'\nLoaded data from extended ion tests\n'

#  FF logistics

FF requires <- pairs_data, data_frame, structures

pickle and do all jax on bach

In [4]:
#python ff_energy/ffe/cli.py -d -m ions_ext -t pbe0dz -e pc -v -msp /home/boittier/pcbach

In [5]:
# %load_ext autoreload
# %autoreload 2

from pathlib import Path
import seaborn as sns
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

from ff_energy.latex_writer.report import Report
from ff_energy.latex_writer.energydata.energy_data_report import EnergyReport
from ff_energy.latex_writer.figure import Figure
from ff_energy.latex_writer.format import safe_latex_string
from ff_energy.utils.ffe_utils import read_from_pickle, pickle_output

from ff_energy.ffe.ff import FF
from ff_energy.ffe.constants import PKL_PATH, PDB_PATH
from ff_energy.ffe.potential import LJ, LJ_bound, DE #, DE_bounds

from ff_energy.ffe.data import pairs_data
from ff_energy.plotting.data_plots import DataPlots



2023-08-31 15:53:01 | DEBUG | __init__:process_initializer_module:72 - Initializing MLIR with module: _site_initialize_0
2023-08-31 15:53:01 | DEBUG | __init__:process_initializer_module:74 - Registering dialects from initializer <module 'jaxlib.mlir._mlir_libs._site_initialize_0' from '/home/boittier/miniconda3/envs/pycharmm/lib/python3.8/site-packages/jaxlib/mlir/_mlir_libs/_site_initialize_0.so'>
2023-08-31 15:53:01 | DEBUG | path:<module>:29 - etils.epath found. Using etils.epath for file I/O.


<Figure size 100x100 with 0 Axes>

In [6]:
"""
Test loading the report from pkl
"""

'\nTest loading the report from pkl\n'

In [7]:
pkl_path = "/home/boittier/Documents/phd/ff_energy/pickles/energy_report.pkl"
er = pd.read_pickle(pkl_path)
# er.add_data()

In [8]:
er.data_plots[0].obj.__dict__.keys()

dict_keys(['system', 'output_path', 'output', 'data', 'coloumb', 'ctot', 'chm_df', 'monomers_df', 'monomer_df', 'cluster_df', 'pairs_df', 'structures', 'pdbs', 'structure_key_pairs', 'min_m_E', 'bonded_fit'])

In [9]:
eg_data = er.data_plots[0].data
# eg_data

# WATER

##  PC

In [10]:
eg_dcm_path_ = "/home/boittier/homeb/water_cluster/pbe0dz_pc/{}/charmm/dcm.xyz"
# dcm_path_="/home/boittier/homeb/water_cluster/pbe0dz_pc/{}/charmm/dcm.xyz",

In [11]:
data = er.data_plots[0].obj

In [12]:
pairs_out = pairs_data(data, 
           system = "water_cluster",
          name = "test",
          dcm_path_ = eg_dcm_path_,
)

2023-08-31 15:55:07 | WARNING | dispatch:log_elapsed_time:270 - Finished tracing + transforming jit(convert_element_type) in 0.000186920166015625 sec
2023-08-31 15:55:07 | DEBUG | xla_bridge:_init_backend:482 - Initializing backend 'interpreter'
2023-08-31 15:55:07 | DEBUG | xla_bridge:_init_backend:494 - Backend 'interpreter' initialized
2023-08-31 15:55:07 | DEBUG | xla_bridge:_init_backend:482 - Initializing backend 'cpu'
2023-08-31 15:55:07 | DEBUG | xla_bridge:_init_backend:494 - Backend 'cpu' initialized
2023-08-31 15:55:07 | DEBUG | xla_bridge:_init_backend:482 - Initializing backend 'cuda'
2023-08-31 15:55:07 | INFO | xla_bridge:backends:450 - Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023-08-31 15:55:07 | DEBUG | xla_bridge:_init_backend:482 - Initializing backend 'rocm'
2023-08-31 15:55:07 | INFO | xla_bridge:backends:450 - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAll

In [14]:
pickle_output(pairs_out, "water_pc_pairs")

#  MDCM

In [ ]:
eg_dcm_path_ = "/home/boittier/homeb/water_cluster/pbe0dz_mdcm/{}/charmm/dcm.xyz"
pairs_out = pairs_data(data, 
           system = "water_cluster",
          name = "mdcm",
          dcm_path_ = eg_dcm_path_,
)

2023-08-31 16:21:26 | WARNING | dispatch:log_elapsed_time:270 - Finished tracing + transforming jit(convert_element_type) in 0.00018668174743652344 sec
2023-08-31 16:21:34 | WARNING | dispatch:log_elapsed_time:270 - Finished tracing + transforming jit(convert_element_type) in 0.00018143653869628906 sec


In [ ]:
pickle_output(pairs_out, "water_mdcm_pairs")

#  pbe0dz_tip3

In [ ]:
eg_dcm_path_ = "/home/boittier/homeb/water_cluster/pbe0dz_tip3/{}/charmm/dcm.xyz"
pairs_out = pairs_data(data, 
           system = "water_cluster",
          name = "tip3",
          dcm_path_ = eg_dcm_path_,
)

In [ ]:
pickle_output(pairs_out, "water_tip3_pairs")

#  kMDCM

In [ ]:
eg_dcm_path_ = "/home/boittier/homeb/water_cluster/pbe0dz_kmdcm/{}/charmm/dcm.xyz"
pairs_out = pairs_data(data, 
           system = "water_cluster",
          name = "kmdcm",
          dcm_path_ = eg_dcm_path_,
)

In [ ]:
pickle_output(pairs_out, "water_kmdcm_pairs")

In [42]:
# dists = read_from_pickle(PKL_PATH / "structures" / "water_cluster.pkl")

In [43]:
# struct, dist = next(dists)

In [56]:
# ?FF

In [1]:
# ff_obj = FF(eg_data, 
#             dist,
#             LJ,
#             LJ_bound,
#             struct[0],
#            )

In [2]:
# pickle_output(ff_obj, "water_pc_ff_orig")